<a href="https://colab.research.google.com/github/Vadiman728/Vadiman728/blob/main/Parkinson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Раздел 1. Импортируем и изучаем датасет (и теория)

In [ ]:
import pandas as pd
url="https://drive.google.com/file/d/1pljfu_FqVFV_o2iGagLP1W75RnehaCX9/view?usp=sharing"
reconstructed_url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(reconstructed_url)
df

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,phon_R01_S50_2,174.188,230.978,94.261,0.00459,0.00003,0.00263,0.00259,0.00790,0.04087,...,0.07008,0.02764,19.517,0,0.448439,0.657899,-6.538586,0.121952,2.657476,0.133050
191,phon_R01_S50_3,209.516,253.017,89.488,0.00564,0.00003,0.00331,0.00292,0.00994,0.02751,...,0.04812,0.01810,19.147,0,0.431674,0.683244,-6.195325,0.129303,2.784312,0.168895
192,phon_R01_S50_4,174.688,240.005,74.287,0.01360,0.00008,0.00624,0.00564,0.01873,0.02308,...,0.03804,0.10715,17.883,0,0.407567,0.655683,-6.787197,0.158453,2.679772,0.131728
193,phon_R01_S50_5,198.764,396.961,74.904,0.00740,0.00004,0.00370,0.00390,0.01109,0.02296,...,0.03794,0.07223,19.020,0,0.451221,0.643956,-6.744577,0.207454,2.138608,0.123306


В информации про датасет написано, что кроме всех этих показателей нам, как программистам нейросетей нужна колонка статус, которая и определяет, здоров человек или нет (1- имеет болезнь Паркинсона, 0 - здоров).

Пользоваться будем XGBoost (eXtreme Gradient Boosting), где основные акценты сделаны на скорости, гибкости и характеристикам модели. Так же огромным преимуществом является оптимизация системы. XGBoost использует тип дерева решений, называемый CART: Classification and Decision Tree (деревья классификации для определения класса и дерево регрессии для прогнозирования)
Еще одной особенностью является использование повышения. Повышение - это тип ансамблевого обучения, в котором результат предыдущей модели используется в качестве входных данных для следующей. Вместо того, чтобы обучать модели по отдельности, ускорение обучающих моделей происходит последовательно, при этом каждая новая модель обучается исправлять ошибки предыдущих. На каждой итерации (раунде) правильно предсказанным результатам присваивается более низкий вес, а ошибочно предсказанным результатам - более высокий вес. Затем он использует средневзвешенное значение для получения окончательного результата.

Для XGBoost есть два типа обучения: Learning API (это базовый низкоуровневый способ использования XGBoost. Простой и мощный, он включает встроенный метод перекрестной проверки) и Scikit_Learn API (это интерфейс-оболочка Scikit-Learn для XGBoost. Он позволяет использовать XGBoost совместимым с scikit-learn способом, точно так же, как вы использовали бы любую нативную модель scikit-learn). При использовании Scikit- Learn также потребуется дополнительно вычислять метрики оценки, когда как learning api делает это "под капотом".

Нам рекомендуют использовать Scikit-learn, что мы и сделаем.

### Раздел 2 Импортируем модель

Вытаскиваем модель из документации, вставляем значения

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from datetime import datetime


X, y = df.drop(['name','status'], axis=1), df['status']
# Дропаем столбик "name" (невалидные данные) и "status" в х, и добавляем его в y

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2)

xgbc = XGBClassifier(objective='reg:squarederror',
                     max_depth = 23,
                     n_estimators = 10500,
                     learning_rate = 0.4,
                     colsample_bytree = 0.8,
                     colsample_bylevel = 0.9
                        )# создаем модель. в теоретической части расписано про каждый параметр
start_time = datetime.now()# ставим секундомер
xgbc.fit(xtrain, ytrain)# запускаем тренировку модели
end_time = datetime.now()

ypred = xgbc.predict(xtest)
mse = mean_squared_error(ytest, ypred) # выводим индекс MSE
print("MSE: ", mse)
print("RMSE: %.2f" % (mse**(1/2.0))) # Рассчитываем RMSE (более информативный)
training_time_tfidf = (end_time - start_time).total_seconds() # Рассчитываем время обучения
print('Training time: {:.5f}s'.format(training_time_tfidf))


from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

y_pred = xgbc.predict(xtest)
y_pred = np.where(y_pred > 0.5, 1, 0) # подгоняем параметры для вывода отчетности


print("classification report: ", classification_report(ytest, y_pred))
print("confusion matrix: ", confusion_matrix(ytest, y_pred))


MSE:  0.02564102564102564
RMSE: 0.16
Training time: 2.98784s
classification report:                precision    recall  f1-score   support

           0       1.00      0.90      0.95        10
           1       0.97      1.00      0.98        29

    accuracy                           0.97        39
   macro avg       0.98      0.95      0.97        39
weighted avg       0.98      0.97      0.97        39

confusion matrix:  [[ 9  1]
 [ 0 29]]



Удалось добиться следующих результатов при настройке параметров внутри самой модели. Так же можно использовать гиперпараметры, но текущей точности вполне хватит

```
MSE:  0.02564102564102564
RMSE: 0.16
Training time: 2.98784s
classification report:                
precision    recall  f1-score   support

           0       1.00      0.90      0.95        10
           1       0.97      1.00      0.98        29

    accuracy                           0.97        39
   macro avg       0.98      0.95      0.97        39
weighted avg       0.98      0.97      0.97        39

confusion matrix:  
[[ 9  1]
 [ 0 29]]
```


Параметр RMSE (Root Mean Squared Error)- квадратный корень из среднеквадратичной ошибки прогноза. Чем меньше этот параметр, тем лучше. Это означает, что чем меньше параметр, тем лучше модель прогнозирует результат (хмм, может попробовать XGBoost прикрутить к анализу финансового рынка...). Если RMSE равен нулю, то это означает, что на всех тестовых фреймах прогнозы точно совпали с реальными значениями.

Параметры, используемые в обучении модели:
* max_depth- глубина дерева для обучения
* n_estimators- количество деревьев в лесу

Чем больше значение двух упомянутых параметров, тем лучше, но тем дольше будет идти обучение. Датасет у нас маленький, потому космических значений этих параметров не требуется.
* learning_rate - повышение скорости обучения
* colsample_bytree - Соотношение подвыборок столбцов при построении каждого дерева.
* colsample_bylevel - Соотношение подвыборок столбцов для каждого уровня.

Нахождение баланса между соотношениями выборок дерева и уровня также дает свой позитивный результат.

На практике чаще дают предпочтение RMSE, нежели MSE (тот же RMSE, но в квадрате), так как RMSE выдается в более читаемом и приземленном виде, в отличие от MSE, где значение может быть 16, 9, 12 и тд.

Удивительная точность достигается предотвращением перетренированности, приоритизаций важных функций (XGBoost устраняет необходимость многократного просмотра и оценки одной и той же функции при построении дерева, поскольку все функции учитываются одновременно) и приближенному жадному алгоритму (При работе с большими наборами данных XGBoost делает алгоритм более масштабируемым и быстрым за счет аппроксимации(нахождения ближнего) оптимального разделения, что значительно снижает вычислительные затраты, связанные с оценкой всех возможных разделений)